<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/financial_formulae_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Financial Formulae**

In [0]:
# Black's formula

from scipy.stats import norm
import math

def black(f_0, strike, expiry, vol, rate, c_p):
    d_plus = (math.log(f_0 / strike) + 0.5 * vol * vol * expiry)/ (vol / math.sqrt(expiry))
    d_minus = (math.log(f_0 / strike) - 0.5 * vol * vol * expiry)/ (vol / math.sqrt(expiry))
    if c_p == 'c':
            option_value = (math.exp((-rate)*expiry))*(f_0 * norm.cdf(d_plus) - strike *norm.cdf(d_minus))
    elif c_p == 'p':
            option_value = (math.exp((-rate)*expiry))*(strike * norm.cdf(-d_minus) - f_0 *norm.cdf(-d_plus))
    else:
            print('Check input')
    return option_value

In [145]:
# Run black_price

# Request inputs
c_p = str(input('call (c) or put (p) = '))

f_0 = 0.07
strike = 0.08
expiry = 1
black_vol = 0.2
rate = 0.05

# Price option
if c_p in ['c', 'p']:
  black_price = black(f_0, strike, expiry, black_vol, rate, c_p)
  print('\n' "black_price = "+"{:.3%}".format(black_price))
else:
  print('Check input')

call (c) or put (p) = p

black_price = 1.165%


In [0]:
# Bachelier's formula 

from scipy.stats import norm
import math

def dPlusBachelier(F_0 , y, expiry , vol):
    d_plus = (F_0 - y)/ (vol * math.sqrt(expiry))
    return d_plus

def bachelier(F_0, y, expiry, vol, rfr, type):
    if expiry * vol == 0.0:
        if type == 'isCall':
            option_value = max(F_0 - y, 0.0)
        elif type == 'isPut':
            option_value = max(y - F_0 , 0.0)
        else:
            option_value = 'Not defined'
    else:
        d1 = dPlusBachelier(F_0, y, expiry, vol)
        if type == 'isCall':
             option_value = (math.exp((-rfr)*expiry))*((F_0 - y)* norm.cdf(d1) + vol * math.sqrt(expiry) * norm.cdf(d1))
        elif type == 'isPut':
             option_value = (math.exp((-rfr)*expiry))*((y-F_0) * norm.cdf(-d1) + vol * math.sqrt(expiry) * norm.cdf(-d1))
        else: option_vale = 'Not defined'
    return option_value

In [141]:
# Run Bachelier 

# Input
F_0_val = 0.07
y_val = 0.08
expiry_val = 2
vol_val = 0.2
rfr_val = 0.07
type_val = 'isCall'   # or 'isPut'

# Output
bachelier_price = bachelier(F_0_val, y_val, expiry_val, vol_val, rfr_val, type_val)
print('bachelier_price = '+'{:.3%}'.format(bachelier_price))

bachelier_price = 11.525%


In [0]:
# Calculate SABR Volatility

import math

def sabr_vol(alpha, beta, rho, v, f, k, T):
  bb = alpha / (f**(1.-beta))
  if k <= 0:                       # For negative rates shift smile
    z = 'n/a'
    x = 'n/a'
    sabr_vol = 0
    note = 'No volatility since negative rates'
  elif f ==  k:
    z = 0
    x = 0
    sabr_vol = bb * (1+ ( (1-beta)*(1-beta)*alpha*alpha/(24*(f**(2-2*beta))) 
    + alpha*beta*rho*v/(4*(f**(1-beta))) + (2-3*rho*rho)*v*v/24) * T )
    note = 'ATM'
  elif f != k:
    z = (v/alpha) * (f*k) ** ((1-beta)/2) * math.log(f/k)
    x = math.log(  ( math.sqrt(1-2*rho*z+z*z) + z - rho ) / (1-rho) )
    denomimator = 1 + ((1-beta)**2)*(1/24) * ((math.log(f/k))**2) 
    + ((1-beta)**4)*(1/1920) * ((math.log(f/k))**4)
    numerator = 1 + ( ((1-beta)**2)*(1/24) * (alpha**2)/((f*k)**(1-beta)) 
    + (1/4)*rho*beta*v*alpha*((f*k)**((1-beta)/2)) 
    + (2-3*rho*rho)*(1/24)*v*v ) * T
    sabr_vol = bb * ((f*k)**((1-beta)/2)) * (z / x) * numerator
    note = 'Not ATM'
  return round(sabr_vol,5), note

In [110]:
# Run SABR calcuation

# Input
alpha_val = 0.2
beta_val = 1
f_val = 0.07
k_val = 0.07
v_val = 0.2
rho_val = -0.3
T_val = 2

# Output
print('SABR', sabr_vol(alpha_val, beta_val, rho_val, v_val, f_val, k_val, T_val))

SABR (0.19995, 'ATM')
